# **Linear-nonlinear-Poisson model – LNPM**

The LNP model is a simplified model that has proven useful to describe the output of single neurons. The steps for  implementing it are described in the following schematic:  

<img src="LNP_flowchart.png">

This diagram serves to outline the steps you'll need to take to make a simulation with the LNP model.

Under this model, a linear filter will determine (partly) the neuron's spiking output by filtering the signal. In very general terms, filtering means letting through what we want while letting out what we don't. Given a neuron, we want signals to go through that are as similar as possible to the neuron's filter, while ignoring signals that differ from it, for the neuron has no business dealing with those. To do the filtering, we should somehow compare the filter with the entire signal. The mathematical operation for this is a convolution, which will "slide" the filter along the signal making a point-to-point multiplication at each step. Similarities will produce larger positive values, whereas discrepancies will produce smaller or negative values. 

The result of filtering the signal with a linear filter by means of convolving is called the generator signal